In [1]:
import argparse
import ipaddress
import os
import networkx as nx

from networkx.algorithms.cluster import average_clustering
from networkx.algorithms.components import *
from networkx.algorithms.dag import (dag_longest_path_length,
									 is_directed_acyclic_graph)
import networkx.algorithms.approximation as approx
from networkx.algorithms.connectivity import node_connectivity, edge_connectivity

from modules import Rule
from utils import construct_graph
import numpy as np

In [2]:
def load_ruleset(fname, except_zero = True):
	""" Load ruleset from ClassBench filter file """
	ruleset = []
	with open(fname, 'r') as f:
		for n, line in enumerate(f):
			# LINE FORMAT
			# @sip_network dip_network sp_low : sp_high dp_low: dp_high protocal/protocol_mask xxx/xxx
			tok = line.strip().split('\t')
			rule = Rule()
			sip = ipaddress.ip_network(tok[0][1:])
			dip = ipaddress.ip_network(tok[1])
			sp = tok[2].split(':')
			dp = tok[3].split(':')
			protocol = tok[4].split('/')
			
			if except_zero:
				if int(sip[0]) == 0 or int(dip[0]) == 0:
					continue
			#sip[0] : given a IP/mask, upper bound of IP address
			#sip[-1] : given a IP/mask, lower bound of IP address
			rule.sip_low = int(sip[0])
			rule.sip_high = int(sip[-1])
			rule.dip_low = int(dip[0])
			rule.dip_high = int(dip[-1])
			rule.sp_low, rule.sp_high = int(sp[0]), int(sp[1])
			rule.dp_low, rule.dp_high = int(dp[0]), int(dp[1])
			rule.protocol_val, rule.protocol_mask = int(protocol[0], 16), int(protocol[1], 16)

			ruleset.append(rule)
	return ruleset

def all_path_lenth(graph):
	paths_len = []
	for idx_s in [idx for idx,i in enumerate(list(dict(G.in_degree()).values())) if (i == 0 and G.out_degree(idx) != 0)]:
		for idx_d in [idx for idx,i in enumerate(list(dict(G.out_degree()).values())) if (i == 0 and G.in_degree(idx) != 0)]:
			if nx.has_path(G,idx_s, idx_d):
				l = list(nx.all_simple_paths(G, source=idx_s, target=idx_d))
				paths_len.append(len(l))
			else:
				continue
	return np.array(paths_len)


In [3]:


""" Load rules from ClassBench filter file, build a graph,
and print graph statistics """

# ruleset = load_ruleset("../output/MyFilters1k_1.exp")
ruleset = load_ruleset("../data/acl filters/MyFilters10k_1.txt")

	# build graph
G = construct_graph(ruleset)
#paths_len = []
#paths_len = all_path_lenth(G)

In [5]:
temp_list = list(nx.weakly_connected_components(G))

In [9]:
for i in temp_list:
    if len(i) == 194:
        t = i

In [10]:
t

{11,
 126,
 319,
 320,
 372,
 387,
 676,
 867,
 886,
 893,
 1073,
 1210,
 1354,
 1410,
 1489,
 1591,
 1763,
 1811,
 1837,
 1848,
 1868,
 1885,
 1905,
 1944,
 1983,
 2049,
 2063,
 2066,
 2067,
 2378,
 2410,
 2473,
 2493,
 2542,
 2599,
 2663,
 2818,
 2858,
 2859,
 2872,
 2873,
 2874,
 2875,
 2876,
 2877,
 2878,
 3018,
 3186,
 3222,
 3283,
 3325,
 3442,
 3578,
 3678,
 3685,
 3709,
 3862,
 4054,
 4133,
 4142,
 4319,
 4351,
 4434,
 4529,
 4532,
 4630,
 4663,
 4683,
 4763,
 4822,
 5007,
 5090,
 5174,
 5180,
 5291,
 5322,
 5364,
 5466,
 5475,
 5479,
 5616,
 5645,
 5744,
 5771,
 5815,
 5817,
 5848,
 5944,
 5957,
 5958,
 5982,
 6000,
 6093,
 6125,
 6178,
 6209,
 6217,
 6253,
 6256,
 6367,
 6419,
 6485,
 6486,
 6487,
 6643,
 6645,
 6667,
 6671,
 6674,
 6755,
 6769,
 6771,
 6857,
 6883,
 6950,
 6952,
 6993,
 7021,
 7027,
 7058,
 7108,
 7109,
 7111,
 7160,
 7175,
 7199,
 7217,
 7236,
 7238,
 7259,
 7296,
 7351,
 7352,
 7362,
 7393,
 7417,
 7419,
 7452,
 7453,
 7454,
 7455,
 7472,
 7473,
 7474,
 74

In [6]:
count = 0
temp_max = 0
for i in temp_list:
    if len(i) > 1:
        count += 1
    if len(i) > temp_max:
        temp_max = len(i)
print(count, temp_max)

240 194


In [ ]:
list(nx.weakly_connected_components(G))

In [ ]:
"""Picture of the graph"""
%matplotlib inline

import matplotlib.pyplot as plt
nx.draw(G, node_size=500, arrows = True, with_labels = True )
plt.show()
plt.savefig('./gragh.pdf')

In [ ]:
"""Traversal of all nodes with 0 in degree and 0 out degree."""

for idx,i in enumerate(list(dict(G.in_degree()).values())):
    for jdx,j in enumerate(list(dict(G.out_degree()).values())):
        if i == 0 and j == 0:
            print(idx,",",jdx)

In [ ]:
print("Number of components is:", len(list(nx.weakly_connected_components(G))))

In [ ]:
"""Small index nodes costs shoterer time(search for a short time)"""
for path in nx.all_simple_paths(G, source=891, target=966):
    print(path)

In [ ]:
"""Big index nodes costs longer time(search for a long time)"""
for path in nx.all_simple_paths(G, source=0, target=4):
    print(path)